## Training GNN with Neighbor Sampling for Node Classification 

> 본 튜토리얼은 `GraphSAGE`를 사용하여 `ogbn-arxiv` 데이터를 기반으로 node classification을 수행합니다. 해당 데이터셋은 매우 큰 스케일의 데이터이기 때문에 Large Graph에서는 어떻게 node classification을 할 수 있을까에 대한 내용을 다루고 있습니다.

In [ ]:
!pip install ogb 

In [ ]:
import dgl 
import torch 
import numpy as np 
from ogb.nodeproppred import DglNodePropPredDataset 

In [4]:
dataset = DglNodePropPredDataset('ogbn-arxiv')

Downloaded 0.08 GB: 100%|██████████| 81/81 [01:37<00:00,  1.21s/it]


Extracting dataset\arxiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<?, ?it/s]


Converting graphs into DGL objects...


100%|██████████| 1/1 [00:00<00:00, 166.75it/s]

Saving...


In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
graph, node_labels = dataset[0]

graph = dgl.add_reverse_edges(graph) # reverse_edges를 통해 directed graph를 undirected graph로 바꾸어 줍니다.
graph.ndata['label'] = node_labels[:, 0] # label을 node 에 입력합니다.
print(graph)
print(node_labels)


node_features = graph.ndata['feat']
num_features = node_features.shape[1]
num_classes = (node_labels.max() + 1).item()
print('Number of classes:', num_classes)

Graph(num_nodes=169343, num_edges=2332486,
      ndata_schemes={'feat': Scheme(shape=(128,), dtype=torch.float32), 'year': Scheme(shape=(1,), dtype=torch.int64), 'label': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={})
tensor([[ 4],
        [ 5],
        [28],
        ...,
        [10],
        [ 4],
        [ 1]])
Number of classes: 40


In [7]:
idx_split = dataset.get_idx_split() # index를 가지고와서 train, valid, test 셋을 분리합니다. 
train_nids = idx_split['train']
valid_nids = idx_split['valid']
test_nids = idx_split['test']

## Defining Neighbor Sampler and Data Loader in DGL

> `DGL`은 minibatch 형태로 사용할 수 있는 generater function을 제공하기 때문에 batch 단위의 데이터만 호출하여 모델을 학습할 수 있습니다. 이때 사용하는 함수가 바로 `dgl.dataloading.DataLoader`입니다. `DataLoader`는 PyTorch에서 사용하는 것과 동일한 방식으로 사용이 가능합니다. Graph에 존재하는 이웃을 Sampling하기 위해서는 `dgl.dataloading.NeighborSampler`를 사용하면 쉽게 Sampling이 가능합니다.

In [8]:
sampler = dgl.dataloading.NeighborSampler([4, 4])

In [13]:
device

'cuda'

In [17]:
sampler = dgl.dataloading.NeighborSampler([4, 4]) # 4명의 이웃을 추출합니다. 
train_dataloader = dgl.dataloading.DataLoader(
    # The following arguments are specific to DGL's DataLoader.
    graph,              # The graph
    train_nids,         # The node IDs to iterate over in minibatches
    sampler,            # The neighbor sampler
    device=device,      # Put the sampled MFGs on CPU or GPU
    # The following arguments are inherited from PyTorch DataLoader.
    batch_size=1024,    # Batch size
    shuffle=True,       # Whether to shuffle the nodes for every epoch
    drop_last=False,    # Whether to drop the last incomplete batch
    num_workers=0       # Number of sampler processes
)

In [18]:
input_nodes, output_nodes, mfgs = example_minibatch = next(iter(train_dataloader)) # batch 단위에 들어간 정보를 출력할 수 있습니다. 
print(example_minibatch)
print("To compute {} nodes' outputs, we need {} nodes' input features".format(len(output_nodes), len(input_nodes)))

[tensor([ 92312, 115166, 151892,  ...,  28142,  16181, 117105]), tensor([ 92312, 115166, 151892,  ..., 167075,  36614, 131277]), [Block(num_src_nodes=13039, num_dst_nodes=4116, num_edges=15033), Block(num_src_nodes=4116, num_dst_nodes=1024, num_edges=3299)]]
To compute 1024 nodes' outputs, we need 13039 nodes' input features


In [19]:
mfg_0_src = mfgs[0].srcdata[dgl.NID] # NID는 고유한 node id를 가지고 오는 것입니다. src는 시작 node를 의미하고, dst는 도착 node를 의미합니다.
mfg_0_dst = mfgs[0].dstdata[dgl.NID]
print(mfg_0_src)
print(mfg_0_dst)
print(torch.equal(mfg_0_src[:mfgs[0].num_dst_nodes()], mfg_0_dst))

tensor([ 92312, 115166, 151892,  ...,  28142,  16181, 117105])
tensor([ 92312, 115166, 151892,  ...,  44630, 121947, 103380])
True


In [20]:
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import SAGEConv

class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, aggregator_type='mean') # aggreator는 mean을 주로 사용합니다. (논문에서 mean을 사용합니다)
        self.conv2 = SAGEConv(h_feats, num_classes, aggregator_type='mean')
        self.h_feats = h_feats

    def forward(self, mfgs, x):
        # Lines that are changed are marked with an arrow: "<---"

        h_dst = x[:mfgs[0].num_dst_nodes()]  # <--- mfg는 message flow graph를 의미합니다. 주변 node들의 정보를 의미합니다. 
        h = self.conv1(mfgs[0], (x, h_dst))  # <---
        h = F.relu(h)
        h_dst = h[:mfgs[1].num_dst_nodes()]  # <---
        h = self.conv2(mfgs[1], (h, h_dst))  # <---
        return h

model = Model(num_features, 128, num_classes).to(device)

# h = self.conv1(g, x)를 입력하게 되면 전체 노드에 대해 계산을 하는 것입니다. 
# 위 코드처럼 h = self.conv1(mfgs[0], (x, h_dst))를 사용하면 sampling된 node에 대해서만 계산하는 것을 의미합니다. 

## Defining Training Loop 

In [21]:
opt = torch.optim.Adam(model.parameters())

In [22]:
valid_dataloader = dgl.dataloading.DataLoader(
    graph, valid_nids, sampler,
    batch_size=1024,
    shuffle=False,
    drop_last=False,
    num_workers=0,
    device=device
)

In [ ]:
import tqdm
import sklearn.metrics

best_accuracy = 0
best_model_path = 'model.pt' # model parameter를 저장할 파일 이름을 의미합니다.
for epoch in range(10):
    model.train()

    with tqdm.tqdm(train_dataloader) as tq:
        for step, (input_nodes, output_nodes, mfgs) in enumerate(tq):
            # feature copy from CPU to GPU takes place here
            inputs = mfgs[0].srcdata['feat']
            labels = mfgs[-1].dstdata['label']

            predictions = model(mfgs, inputs)

            loss = F.cross_entropy(predictions, labels)
            opt.zero_grad()
            loss.backward()
            opt.step()

            accuracy = sklearn.metrics.accuracy_score(labels.cpu().numpy(), predictions.argmax(1).detach().cpu().numpy())

            tq.set_postfix({'loss': '%.03f' % loss.item(), 'acc': '%.03f' % accuracy}, refresh=False)

    model.eval()

    predictions = []
    labels = []
    with tqdm.tqdm(valid_dataloader) as tq, torch.no_grad(): # torch.no_grad()를 사용하여야 기울기를 계산하지 않습니다.
        for input_nodes, output_nodes, mfgs in tq:
            inputs = mfgs[0].srcdata['feat']
            labels.append(mfgs[-1].dstdata['label'].cpu().numpy())
            predictions.append(model(mfgs, inputs).argmax(1).cpu().numpy())
        predictions = np.concatenate(predictions)
        labels = np.concatenate(labels)
        accuracy = sklearn.metrics.accuracy_score(labels, predictions)
        print('Epoch {} Validation Accuracy {}'.format(epoch, accuracy))
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), best_model_path)

        # Note that this tutorial do not train the whole model to the end.
        break